# Test Samplers by Hopping on a Graph

Let's test samplers by checking whether they run a model the same way. The model is that a particle hops around a graph, from graph node to graph node. Each hop is determined by a continuous univariate distribution. We'll let a particle hop around and then ask, at the end, how much time it spent on each node.

In [1]:
using Pkg
Pkg.activate(".")
using Fleck
using LinearAlgebra
using Revise

  Activating project at `~/dev/Fleck/examples`


In [2]:
include("../test/graph_occupancy.jl")

sample_run_graph_occupancy (generic function with 1 method)

Make the model. This model is made by a random number generator, where the random number generator determines which nodes are connected (so you can hop from one to the other) and which distribution determines the speed of each hop.

Our plan is to use bootstrapping to estimate how much variance we should expect with different samplers. We'll start with a sampler we trust and run this 100 times. Then assume the average result is our expected value. We look at the odds ratio of those 100 draws to see how big they can get. Then, when we want to test another sampler, run it 10 or 100 times and look to see if its odds ratios are mostly within the range that we've seen already.

In [3]:
Threads.nthreads()

24

In [3]:
model_rng = Xoshiro(3498217)  # Random number generator.
singleton_groc = GraphOccupancy(model_rng)
length(singleton_groc)

8

In [4]:
function generate_samples(singleton_groc, singleton_sampler, trial_cnt)
    occupancy = zeros(Float64, (length(singleton_groc), trial_cnt))
    rng = Xoshiro(24370233)
    Threads.@threads for trial_idx in 1:trial_cnt
        groc = deepcopy(singleton_groc)
        sampler = deepcopy(singleton_sampler)
        occ, observations = run_graph_occupancy(groc, 1e6, sampler, rng)
        occupancy[:, trial_idx] .= occ
    end
    occupancy
end

generate_samples (generic function with 1 method)

In [5]:
sampler = FirstReaction{keyspace(GraphOccupancy)}()
trial_cnt = 100
occupancy = generate_samples(singleton_groc, sampler, trial_cnt)

8×100 Matrix{Float64}:
    2.29752e5     2.29604e5       2.29538e5  …     2.29357e5     2.29427e5
    1.61046e5     1.61346e5  161444.0              1.60727e5     1.61009e5
    0.0           0.0             0.0              0.0           0.0
 1513.03       1590.13         1601.87          1440.34       1380.28
    1.83855e5     1.83686e5       1.83997e5        1.83725e5     1.84079e5
    1.76578e5     1.76796e5       1.76482e5  …     1.77218e5     1.76748e5
    1.01639e5     1.01367e5       1.01653e5        1.0169e5      1.0162e5
    1.45617e5     1.4561e5        1.45283e5        1.45844e5     1.45737e5

That zero dwell time is pesky. It means there was a component of the graph that wasn't connected, which is fine, but it will throw off the stats. We're going to see how likely each draw is using https://en.wikipedia.org/wiki/Multinomial_test.

In [6]:
function occupancy_likelihood_ratio(occupancy, mle)
    # Sometimes a graph node is disconnected, so remove those.
    keep_row = [sum(occupancy[check_idx,:]) > 0 for check_idx in 1:size(occupancy, 1)]
    occupancy = occupancy[keep_row, :]
    category_cnt = size(occupancy, 1)
    trial_cnt = size(occupancy, 2)
    odds_ratio = zeros(Float64, trial_cnt)
    for trial_idx in 1:trial_cnt
        relative_occupancy = occupancy[:, trial_idx] / sum(occupancy[:, trial_idx])
        log_odds = 0.0
        for node_idx in 1:category_cnt
            log_odds += occupancy[node_idx, trial_idx] * log(mle[node_idx] / relative_occupancy[node_idx])
        end
        odds_ratio[trial_idx] = -2 * log_odds
    end
    return odds_ratio
end

function maximum_likelihood(occupancy)
    # Sometimes a graph node is disconnected, so remove those.
    keep_row = [sum(occupancy[check_idx,:]) > 0 for check_idx in 1:size(occupancy, 1)]
    occupancy = occupancy[keep_row, :]
    category_cnt = size(occupancy, 1)
    trial_cnt = size(occupancy, 2)
    expected_occupancy = sum(occupancy; dims=2) / trial_cnt
    mle = expected_occupancy / sum(expected_occupancy)
end

maximum_likelihood (generic function with 1 method)

This set of 100 numbers is 100 odds ratios. If we do a bunch of draws from another sampler, we expect their odds ratios to be mostly in this range.

In [7]:
mle = maximum_likelihood(occupancy)
odds_ratios = occupancy_likelihood_ratio(occupancy, mle)
sort!(odds_ratios)

100-element Vector{Float64}:
   0.49345455918449943
   0.7941154347736727
   1.3462359633462597
   1.3960414673955768
   1.5440295841996203
   1.5678766070710992
   1.5750898709526666
   1.8536652543081118
   1.9829543509065388
   2.0602998227698777
   ⋮
  13.663313791737664
  13.798581498385033
  14.179756557389737
  17.54736500691581
  22.801953752270265
  61.346576818131894
  74.85372854513116
  94.48419734065669
 108.65127090668057

Just checking that a perfect odds ratio is basically zero.

In [8]:
perfect = zeros(length(mle), 2)
perfect[:,1] .= mle * 1e6
perfect[:,2] .= mle * 1e6
occupancy_likelihood_ratio(perfect, mle)

2-element Vector{Float64}:
 -1.8427170534944764e-10
 -1.8427170534944764e-10

Now we're ready to evaluate the FirstToFire sampler. This should behave identically to the FirstReaction sampler, especially considering that this model disables all enabled nodes at every step.

In [35]:
sampler = FirstToFire{keyspace(GraphOccupancy)}()
trial_cnt = 100
first_to_fire = generate_samples(singleton_groc, sampler, trial_cnt)

Expected old time 525859.0884321863 to be less than new time 525859.0884321863
Expected old time 529726.7188908086 to be less than new time 529726.7188908086
Transition was Exponential{Float64}(θ=0.919189636064143)
Transition shift 0.0
Transition memory false
Transition consumed 0.0
Transition was Exponential{Float64}(θ=0.9540092435974449)
Transition shift 0.0
Transition memory false
Transition consumed 0.0


8×100 Matrix{Float64}:
    2.28308e5       2.27544e5  …       2.2803e5      2.27459e5
    1.59352e5       1.59931e5     159902.0           1.60343e5
    0.0             0.0                0.0           0.0
 1470.22         1504.78            1501.53       1427.81
    1.82363e5       1.82732e5          1.82633e5     1.83229e5
    1.75896e5  175329.0        …       1.76213e5     1.74845e5
    1.05245e5       1.05091e5          1.04793e5     1.04964e5
    1.47367e5       1.47868e5          1.46927e5     1.47732e5

Well, these odds don't look like they are in the right range.

In [36]:
ftf_odds_ratio = occupancy_likelihood_ratio(first_to_fire, mle)
sort!(ftf_odds_ratio)

100-element Vector{Float64}:
 108.80038192151687
 115.62483626228914
 124.02258491331668
 124.38059189901787
 125.95866491687502
 127.52001715783945
 127.79860457100949
 129.69460152168585
 141.13872019356677
 145.2221799682875
   ⋮
 216.70654833379012
 222.50752856145346
 223.5938911574658
 224.17995241333392
 226.84122816242416
 233.59314277986232
 250.32552679576656
 281.05655277413007
 286.0982583300356

Let's look in more depth at a single run with this sampler. We were using the total duration spent on each node during $10^6$ time units in order to evaluate if the sampler is working well. We can dig in by looking at another output of the model. The "observations" are a dictionary with an entry for each transition that tells us the total number of times that transition fired. If we group them by source node, that tells us how the particle left each node. If one type of transition wasn't firing properly due to some code error, it should show up as a bias in how the particle leaves the node.

In [22]:
rng = Xoshiro(2431243)
groc_ftf = deepcopy(singleton_groc)
sampler_ftf = FirstToFire{keyspace(GraphOccupancy)}()
occupancy_ftf, observations_ftf = run_graph_occupancy(groc_ftf, 1e7, sampler_ftf, rng)
groc_fr = deepcopy(singleton_groc)
sampler_fr = FirstReaction{keyspace(GraphOccupancy)}()
occupancy_fr, observations_fr = run_graph_occupancy(groc_fr, 1e7, sampler_fr, rng)


([2.31180496613119e6, 1.595318722941517e6, 0.0, 14263.47604591521, 1.8054266318270252e6, 1.787476771794091e6, 1.0183025283164126e6, 1.4674067812774011e6], Dict{Tuple{Int64, Int64}, TransitionObserver}((6, 8) => TransitionObserver(2.224929630756378e-6, 5.345915071666241, 0.4537133043736416, 405289.3908244516, 893272), (4, 5) => TransitionObserver(5.442136898636818e-5, 8.550146629102528, 1.002282063517336, 14263.47604591521, 14231), (2, 5) => TransitionObserver(7.729977369308472e-8, 6.9354165319819, 0.604621289751055, 1.0491019800773559e6, 1735139), (6, 2) => TransitionObserver(0.5691427085548639, 5.955648178234696, 0.9585710047237068, 423438.1970556455, 441739), (2, 6) => TransitionObserver(0.0009621046483516693, 6.907002249732614, 0.9138334472473781, 459827.28315317194, 503185), (7, 8) => TransitionObserver(3.9674341678619385e-7, 4.564894130453467, 0.33253663383340687, 352258.38397616474, 1059307), (7, 2) => TransitionObserver(0.2611461281776428, 3.3831943217664957, 1.3608218706302526,

Just looking at the raw occupancies shows that the total dwell time on node 4 could be off. Because this is a particle on a graph, you can see dwell time increase on one node because of a problem on a neighboring node that dumps out to it. That is, this high dwell time doesn't mean there is a problem with the distributions on that node. We'll have to dig a little deeper.

In [23]:
hcat(occupancy_fr, occupancy_ftf)

8×2 Matrix{Float64}:
     2.3118e6       2.31296e6
     1.59532e6      1.59358e6
     0.0            0.0
 14263.5        14070.7
     1.80543e6      1.80624e6
     1.78748e6      1.78737e6
     1.0183e6       1.01799e6
     1.46741e6      1.4678e6

In [24]:
sort(collect(observations_fr), by = a -> a[1][1])

24-element Vector{Pair{Tuple{Int64, Int64}, TransitionObserver}}:
 (1, 5) => TransitionObserver(0.0004142913967370987, 3.3154873268213123, 0.7184295751620818, 1.4606794013182377e6, 2033156)
 (1, 6) => TransitionObserver(0.21658028103411198, 3.0526506998576224, 0.5954087985870061, 851125.5648129521, 1429481)
 (2, 5) => TransitionObserver(7.729977369308472e-8, 6.9354165319819, 0.604621289751055, 1.0491019800773559e6, 1735139)
 (2, 6) => TransitionObserver(0.0009621046483516693, 6.907002249732614, 0.9138334472473781, 459827.28315317194, 503185)
 (2, 7) => TransitionObserver(0.00010904204100370407, 6.290487241465598, 0.915152276093912, 86389.4597109892, 94399)
 (4, 5) => TransitionObserver(5.442136898636818e-5, 8.550146629102528, 1.002282063517336, 14263.47604591521, 14231)
 (5, 6) => TransitionObserver(1.2330710887908936e-6, 3.0050602620467544, 0.31383245505469204, 56596.85877701821, 180341)
 (5, 1) => TransitionObserver(2.598389983177185e-7, 3.1215849025174975, 0.30532290833075487, 51152

In [25]:
sort(collect(observations_ftf), by = a -> a[1][1])

24-element Vector{Pair{Tuple{Int64, Int64}, TransitionObserver}}:
 (1, 5) => TransitionObserver(0.00013344455510377884, 3.4860272853984497, 0.7192650482879202, 1.4610955989357773e6, 2031373)
 (1, 6) => TransitionObserver(0.2165798507630825, 3.255967485718429, 0.5957999838047995, 851861.1134444748, 1429777)
 (2, 5) => TransitionObserver(3.5390257835388184e-8, 7.179546479135752, 0.6044925132029497, 1.048277064815816e6, 1734144)
 (2, 6) => TransitionObserver(0.0012898272834718227, 6.761778498068452, 0.9128762516626613, 459243.39455269, 503073)
 (2, 7) => TransitionObserver(8.825212717056274e-6, 6.503641169518232, 0.9122680860444093, 86056.98536082727, 94333)
 (4, 5) => TransitionObserver(0.00040536653250455856, 9.742169450037181, 1.0106061327827656, 14070.669186734445, 13923)
 (5, 6) => TransitionObserver(1.3532117009162903e-6, 3.060934967827052, 0.31297677441042165, 56536.12452949857, 180640)
 (5, 1) => TransitionObserver(4.807952791452408e-8, 3.2856816863641143, 0.30567179978108466, 511

Here, I'm trying to summarize, node-by-node, the probability of leaving that node for a neighbor. So, given that you are at node 5, you can go to nodes 4, 6, 7, 2, 8, or 1. Let's take every transitition from 5 to each of those, normalize by the each direction, and turn it into a probability for P[4|5], P[6|5], and so on. Do this for both the FirstReaction and FirstToFire and compare the tuples.

In [26]:
ways_out = Array{Dict{Int,Tuple{Float64,Float64}},1}()
node_cnt = nv(singleton_groc.g)
for init_ways in 1:node_cnt
    comparator = Dict{Int,Tuple{Float64,Float64}}()
    totals = zeros(Int64, 0)
    for obs in [observations_fr, observations_ftf]
        total_calls = 0
        for jump_to in neighbors(singleton_groc.g, init_ways)
            total_calls += obs[(init_ways, jump_to)].call_cnt
        end
        push!(totals, total_calls)
    end
    for way_out in neighbors(singleton_groc.g, init_ways)
        comparator[way_out] = (observations_fr[(init_ways, way_out)].call_cnt / totals[1],
            observations_ftf[(init_ways, way_out)].call_cnt / totals[2])
    end
    push!(ways_out, comparator)
end
ways_out

8-element Vector{Dict{Int64, Tuple{Float64, Float64}}}:
 Dict(5 => (0.5871698361682152, 0.5869069528913801), 6 => (0.41283016383178484, 0.41309304710862))
 Dict(5 => (0.7438255635152566, 0.7437730265274174), 6 => (0.21570713710972114, 0.2157676223971178), 7 => (0.04046729937502224, 0.04045935107546482))
 Dict()
 Dict(5 => (1.0, 1.0))
 Dict(4 => (0.0026944603856007402, 0.0026372674704053245), 6 => (0.034145294104393445, 0.03421647603634402), 7 => (0.25375186236989444, 0.2538830699664806), 2 => (0.3579985674733134, 0.3581423620582013), 8 => (0.03420133797159447, 0.03416097654853042), 1 => (0.31720847769520355, 0.3169598479200383))
 Dict(5 => (0.07502530896485297, 0.07480054028702526), 7 => (0.00020199782363635179, 0.00018191282820229535), 2 => (0.1308362413714053, 0.13054141583287485), 8 => (0.2645733136587848, 0.26479160167134635), 1 => (0.5293631381813206, 0.5296845293805512))
 Dict(5 => (0.3212420982211688, 0.3215274283660601), 6 => (0.33266913530563125, 0.33224327768157613), 2 => (6.

Actually, that node 4 only has a single path to another node. Let's see what the transition away is. It does have a relative_enabling_time, meaning it always claims to have a te 0.9 time units in the past.

In [27]:
singleton_groc.transition[(4,5)]

Transition(Exponential{Float64}(θ=1.0025218240393494), -0.9162202776014298, false, 0.0)

And in the other direction there is also an offset, this time into the future. This one is more suspicious, actually.

In [28]:
singleton_groc.transition[(5,4)] 

Transition(Gamma{Float64}(α=2.0, θ=2.0), 0.5815720074869544, false, 0.0)

In [34]:
for dest in [1,2,4,6,7,8]
    print("fr  ")
    println(observations_fr[(5,dest)])
    print("ftf ")
    println(observations_ftf[(5,dest)])
end

fr  TransitionObserver(2.598389983177185e-7, 3.1215849025174975, 0.30532290833075487, 511526.0930239218, 1675361)
ftf TransitionObserver(4.807952791452408e-8, 3.2856816863641143, 0.30567179978108466, 511491.3210866813, 1673335)
fr  TransitionObserver(1.1082738637924194e-7, 3.160241512581706, 0.30479267458294546, 576301.0747234095, 1890797)
ftf TransitionObserver(2.468004822731018e-8, 3.275742782279849, 0.30533464030926066, 577311.7764993749, 1890751)
fr  TransitionObserver(0.5849626380950212, 3.1341096609830856, 1.0365169941186374, 14750.67334330233, 14231)
ftf TransitionObserver(0.582367455586791, 3.1338147297501564, 1.03975434225738, 14476.499707249503, 13923)
fr  TransitionObserver(1.2330710887908936e-6, 3.0050602620467544, 0.31383245505469204, 56596.85877701821, 180341)
ftf TransitionObserver(1.3532117009162903e-6, 3.060934967827052, 0.31297677441042165, 56536.12452949857, 180640)
fr  TransitionObserver(3.3010728657245636e-5, 3.4024405404925346, 0.4397761422131869, 589392.383555535

In [30]:
observations_ftf[(5,4)]

TransitionObserver(0.582367455586791, 3.1338147297501564, 1.03975434225738, 14476.499707249503, 13923)